In [1]:
import os
import re
import sys
import json
import codecs
import locale
import logging
import requests
from enum import Enum
import datetime as dt
from typing import Union, Optional, Tuple, List, cast

from matplotlib import pyplot as plt    # type: ignore  
import matplotlib.gridspec as gridspec  # type: ignore 

import pandas as pd
import numpy as np

In [2]:
#----------------------------------------------------------------
# Configurations section
#----------------------------------------------------------------

temp_content_dir = os.path.join(os.sep, 'tmp')
images_path = os.path.join("..","images")

now = dt.datetime.now()
sample_date = now.strftime("%d/%m/%Y")

locale.setlocale(locale.LC_ALL, 'it_IT.UTF-8')

ok_statuses = [200, 201, 202]
data_file_path = os.path.join(os.sep,"Users","ERIZZAG5J","Work","jmeter")
data_file_name = "VAS-manifest-akamai_20220216_080823.jtl"
csv_data_file = os.path.join(data_file_path, data_file_name)

image_file_prefix = "Test-"

class TypeOfOutputChart(Enum):
    PDF = 1
    PNG = 2
    JPG = 3

output_chart_type = TypeOfOutputChart.JPG

In [3]:
df = pd.read_csv(csv_data_file
                ,sep='#')  
(df.shape[0],df.dtypes)

(37005,
 timeStamp            int64
 elapsed              int64
 label               object
 responseCode         int64
 responseMessage     object
 threadName          object
 dataType            object
 success               bool
 failureMessage      object
 bytes                int64
 sentBytes            int64
 grpThreads           int64
 allThreads           int64
 URL                 object
 Filename           float64
 Latency              int64
 SampleCount          int64
 ErrorCount           int64
 Hostname            object
 IdleTime             int64
 Connect              int64
 dtype: object)

In [5]:
#hit = df.groupby(["CACHE_HIT"]).sum()
#hit

In [6]:
df.loc[:,["threadName","grpThreads","allThreads"]]

,threadName,grpThreads,allThreads
0,setUp Thread Group 1-1,1,1
1,setUp Thread Group 1-1,1,1
2,Manifest 1-1,1,1
3,Manifest 1-1,1,1
4,Manifest 1-1,1,1
...,...,...,...
37000,Manifest 1-32,3,3
37001,Manifest 1-12,3,3
37002,Manifest 1-17,1,1
37003,tearDown Thread Group 1-1,1,1


In [7]:
df.loc[:,"threadName"].unique()

array(['setUp Thread Group 1-1', 'Manifest 1-1', 'Manifest 1-2',
       'Manifest 1-3', 'Manifest 1-4', 'Manifest 1-5', 'Manifest 1-6',
       'Manifest 1-7', 'Manifest 1-8', 'Manifest 1-9', 'Manifest 1-10',
       'Manifest 1-11', 'Manifest 1-12', 'Manifest 1-13', 'Manifest 1-14',
       'Manifest 1-15', 'Manifest 1-16', 'Manifest 1-17', 'Manifest 1-18',
       'Manifest 1-19', 'Manifest 1-20', 'Manifest 1-21', 'Manifest 1-22',
       'Manifest 1-23', 'Manifest 1-24', 'Manifest 1-25', 'Manifest 1-26',
       'Manifest 1-27', 'Manifest 1-28', 'Manifest 1-29', 'Manifest 1-30',
       'Manifest 1-31', 'Manifest 1-32', 'Manifest 1-33', 'Manifest 1-34',
       'Manifest 1-35', 'tearDown Thread Group 1-1'], dtype=object)